In [ ]:
import numpy as np

X = np.array(
    [[169, 70], [172, 68], [175, 78], [163, 58], [180, 80], [159, 76], [158, 52], [173, 69], [180, 75], [155, 50],
     [187, 90], [170, 66]])

m = np.mean(X, axis=0)
cv = np.cov(X, rowvar=False)

gen = np.random.multivariate_normal(m, cv, 5)

print(gen)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
X = x_train[np.isin(y_train, [0])]
X = X.reshape((X.shape[0], 28 * 28))

m = np.mean(X, axis=0)
cv = np.cov(X, rowvar=False)

gen = np.random.multivariate_normal(m, cv, 5)

plt.figure(figsize=(10, 4))
for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.imshow(gen[i].reshape((28, 28)), cmap='gray')
    plt.xticks([])
    plt.yticks([])

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.mixture import GaussianMixture

(x_train, y_train), (x_test, y_test) = mnist.load_data()
X = x_train[np.isin(y_train, [0])]
X = X.reshape((X.shape[0], 28 * 28))

k = 8

gm = GaussianMixture(n_components=k).fit(X)

gen = gm.sample(n_samples=10)

import matplotlib.pyplot as plt

plt.figure(figsize=(20, 4))
for i in range(k):
    plt.subplot(1, 10, i + 1)
    plt.imshow(gm.means_[i].reshape((28, 28)), cmap='gray')
    plt.xticks([])
    plt.yticks([])
plt.show()

plt.figure(figsize=(20, 4))
for i in range(10):
    plt.subplot(1, 10, i + 1)
    plt.imshow(gen[0][i].reshape((28, 28)), cmap='gray')
    plt.xticks([])
    plt.yticks([])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Model

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

zdim = 32

encoder_input = Input(shape=(28, 28, 1))
x = Conv2D(32, (3, 3), activation='relu', padding='same', strides=(1, 1))(encoder_input)
x = Conv2D(64, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', strides=(1, 1))(x)
x = Flatten()(x)
encoder_output = Dense(zdim)(x)
model_encoder = Model(encoder_input, encoder_output)

decoder_input = Input(shape=(zdim,))
x = Dense(3136)(decoder_input)
x = Reshape((7, 7, 64))(x)
x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same', strides=(1, 1))(x)
x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
x = Conv2DTranspose(32, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
x = Conv2DTranspose(1, (3, 3), activation='relu', padding='same', strides=(1, 1))(x)
decoder_output = x
model_decoder = Model(decoder_input, decoder_output)

model_input = encoder_input
model_output = model_decoder(encoder_output)
model = Model(model_input, model_output)

model.compile(optimizer='Adam', loss='mse')
model.fit(x_train, x_train, epochs=50, batch_size=128, shuffle=True, validation_data=(x_test, x_test))

i = np.random.randint(x_test.shape[0])
j = np.random.randint(x_test.shape[0])
x = np.array((x_test[i], x_test[j]))
z = model_encoder.predict(x)

zz = np.zeros((11, zdim))
alpha = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
for i in range(11):
    zz[i] = (1.0 - alpha[i]) * z[0] + alpha[i] * z[1]

gen = model_decoder.predict(zz)

plt.figure(figsize=(20, 4))
for i in range(11):
    plt.subplot(1, 11, i + 1)
    plt.imshow(gen[i].reshape(28, 28), cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.title(str(alpha[i]))

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Conv2D, Conv2DTranspose, Lambda
from tensorflow.keras.models import Model
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from keras import backend as K

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

zdim = 32


def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], zdim), mean=0.0, stddev=0.1)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


encoder_input = Input(shape=(28, 28, 1))
x = Conv2D(32, (3, 3), activation='relu', padding='same', strides=(1, 1))(encoder_input)
x = Conv2D(64, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', strides=(1, 1))(x)
x = Flatten()(x)
z_mean = Dense(zdim)(x)
z_log_var = Dense(zdim)(x)
encoder_output = Lambda(sampling)([z_mean, z_log_var])
model_encoder = Model(encoder_input, [z_mean, z_log_var, encoder_output])

decoder_input = Input(shape=(zdim,))
x = Dense(3136)(decoder_input)
x = Reshape((7, 7, 64))(x)
x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same', strides=(1, 1))(x)
x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
x = Conv2DTranspose(32, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
x = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same', strides=(1, 1))(x)
decoder_output = x
model_decoder = Model(decoder_input, decoder_output)

model_input = encoder_input
model_output = model_decoder(encoder_output)
model = Model(model_input, model_output)

reconstruction_loss = tf.reduce_mean(
    tf.reduce_sum(keras.losses.binary_crossentropy(model_input, model_output), axis=(1, 2)))
kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
total_loss = reconstruction_loss + kl_loss
model.add_loss(total_loss)

model.compile(optimizer='adam')
model.fit(x_train, x_train, epochs=50, batch_size=128, validation_data=(x_test, x_test))

i = np.random.randint(x_test.shape[0])
j = np.random.randint(x_test.shape[0])
x = np.array((x_test[i], x_test[j]))
z = model_encoder.predict(x)[2]

zz = np.zeros((11, zdim))
alpha = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
for i in range(11):
    zz[i] = (1.0 - alpha[i]) * z[0] + alpha[i] * z[1]

gen = model_decoder.predict(zz)

plt.figure(figsize=(20, 4))
for i in range(11):
    plt.subplot(1, 11, i + 1)
    plt.imshow(gen[i].reshape(28, 28), cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.title(str(alpha[i]))

In [ ]:
import numpy as np
from keras.datasets import fashion_mnist
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
import matplotlib.pyplot as plt

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train[np.isin(y_train, [8])]
x_train = (x_train.astype('float32') / 255.0) * 2.0 - 1.0

zdim = 100


def make_discriminator(in_shape):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same', activation=LeakyReLU(alpha=0.2), input_shape=in_shape))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same', activation=LeakyReLU(alpha=0.2)))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same', activation=LeakyReLU(alpha=0.2)))
    model.add(Conv2D(256, (3, 3), strides=(2, 2), padding='same', activation=LeakyReLU(alpha=0.2)))
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])
    return model


def make_generator(zdim):
    model = Sequential()
    model.add(Dense(7 * 7 * 64, activation=LeakyReLU(alpha=0.2), input_dim=zdim))
    model.add(Reshape((7, 7, 64)))
    model.add(Conv2D(128, (3, 3), padding='same', activation=LeakyReLU(alpha=0.2)))
    model.add(
        Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', activation=LeakyReLU(alpha=0.2)))
    model.add(
        Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', activation=LeakyReLU(alpha=0.2)))
    model.add(Conv2D(1, (3, 3), padding='same', activation='tanh'))
    return model


def make_gan(G, D):
    D.trainable = False
    model = Sequential()
    model.add(G)
    model.add(D)
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
    return model


def generate_real_samples(dataset, n_samples):
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    x = dataset[ix]
    y = np.ones((n_samples, 1))
    return x, y


def generate_latent_points(zdim, n_samples):
    return np.random.randn(n_samples, zdim)


def generate_fake_samples(G, zdim, n_samples):
    x_input = generate_latent_points(zdim, n_samples)
    x = G.predict(x_input)
    y = np.zeros((n_samples, 1))
    return x, y


def train(G, D, GAN, dataset, zdim, n_epochs=200, batch_siz=128, verbose=0):
    n_batch = int(dataset.shape[0] / batch_siz)

    for epoch in range(n_epochs):
        for b in range(n_batch):
            x_real, y_real = generate_real_samples(dataset, batch_siz // 2)
            d_loss1, _ = D.train_on_batch(x_real, y_real)
            x_fake, y_fake = generate_fake_samples(G, zdim, batch_siz // 2)
            d_loss2, _ = D.train_on_batch(x_fake, y_fake)

            x_gan = generate_latent_points(zdim, batch_siz)
            y_gan = np.ones((batch_siz, 1))
            g_loss = GAN.train_on_batch(x_gan, y_gan)
        if verbose == 1:
            print('E%d : loss D(real) = %.3f, D(fake)%.3f GAN %.3f' % (epoch + 1, d_loss1, d_loss2, g_loss))
        if (epoch + 1) % 10 == 0:
            x_fake, y_fake = generate_fake_samples(G, zdim, 12)
            plt.figure(figsize=(20, 2))
            plt.suptitle('epoch ' + str(epoch + 1))
            for k in range(12):
                plt.subplot(1, 12, k + 1)
                plt.imshow((x_fake[k] + 1) / 2.0, cmap='gray')
                plt.xticks([])
                plt.yticks([])
            plt.show()


D = make_discriminator((28, 28, 1))
G = make_generator(zdim)
GAN = make_gan(G, D)
train(G, D, GAN, x_train, zdim, verbose=1)

In [ ]:
import numpy as np
from keras.datasets import cifar10
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
import matplotlib.pyplot as plt

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train[np.isin(y_train.reshape([y_train.shape[0]]), [1])]
x_train = (x_train.astype('float32') / 255.0) * 2.0 - 1.0

zdim = 100


def make_discriminator(in_shape):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same', activation=LeakyReLU(alpha=0.2), input_shape=in_shape))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same', activation=LeakyReLU(alpha=0.2)))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same', activation=LeakyReLU(alpha=0.2)))
    model.add(Conv2D(256, (3, 3), strides=(2, 2), padding='same', activation=LeakyReLU(alpha=0.2)))
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])
    return model


def make_generator(zdim):
    model = Sequential()
    model.add(Dense(4 * 4 * 256, activation=LeakyReLU(alpha=0.2), input_dim=zdim))
    model.add(Reshape((4, 4, 256)))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', activation=LeakyReLU(alpha=0.2)))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', activation=LeakyReLU(alpha=0.2)))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', activation=LeakyReLU(alpha=0.2)))
    model.add(Conv2D(3, (3, 3), padding='same', activation='tanh'))
    return model


def make_gan(G, D):
    D.trainable = False
    model = Sequential()
    model.add(G)
    model.add(D)
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
    return model


def generate_real_samples(dataset, n_samples):
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    x = dataset[ix]
    y = np.ones((n_samples, 1))
    return x, y


def generate_latent_points(zdim, n_samples):
    return np.random.randn(n_samples, zdim)


def generate_fake_samples(G, zdim, n_samples):
    x_input = generate_latent_points(zdim, n_samples)
    x = G.predict(x_input)
    y = np.zeros((n_samples, 1))
    return x, y


def train(G, D, GAN, dataset, zdim, n_epochs=200, batch_siz=128, verbose=0):
    n_batch = int(dataset.shape[0] / batch_siz)

    for epoch in range(n_epochs):
        for b in range(n_batch):
            x_real, y_real = generate_real_samples(dataset, batch_siz // 2)
            d_loss1, _ = D.train_on_batch(x_real, y_real)
            x_fake, y_fake = generate_fake_samples(G, zdim, batch_siz // 2)
            d_loss2, _ = D.train_on_batch(x_fake, y_fake)

            x_gan = generate_latent_points(zdim, batch_siz)
            y_gan = np.ones((batch_siz, 1))
            g_loss = GAN.train_on_batch(x_gan, y_gan)
        if verbose == 1:
            print('E%d : loss D(real) = %.3f, D(fake)%.3f GAN %.3f' % (epoch + 1, d_loss1, d_loss2, g_loss))
        if (epoch + 1) % 10 == 0:
            x_fake, y_fake = generate_fake_samples(G, zdim, 12)
            plt.figure(figsize=(20, 2))
            plt.suptitle('epoch ' + str(epoch + 1))
            for k in range(12):
                plt.subplot(1, 12, k + 1)
                plt.imshow((x_fake[k] + 1) / 2.0, cmap='gray')
                plt.xticks([])
                plt.yticks([])
            plt.show()


D = make_discriminator((32, 32, 3))
G = make_generator(zdim)
GAN = make_gan(G, D)
train(G, D, GAN, x_train, zdim, verbose=1)

In [ ]:
import keras_cv
import matplotlib.pyplot as plt

model = keras_cv.models.StableDiffusion(img_width=512, img_height=512)
img = model.text_to_image('A cute rabbit in an avocado armchair', batch_size=3)

for i in range(len(img)):
    plt.subplot(1, len(img), i + 1)
    plt.imshow(img[i])
    plt.axis('off')